# Your First Model - Fashion MNIST

## Fashion MNIST Dataset

Each picture is 28x28 pixels or 784 bytes

| Label | Class |
|-------|-------|
| 0 | T-Shirt/Top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandals |
| 6 | Shirt |
| 7 | Sneakers |
| 8 | Bag |
| 9 | Ankle Boot |

The dataset has 70,000 images, 60,000 will be used for training and the remaining 10,000 will be used for testing.

## Neural Network

The input ot the network will be the 784 bytes, or pixels

Flattening - Taking a 2D image and converting it to a 1D vector

The dense layer is 128 units, and will be fully connected to the previous. It also has a ReLU activation because this is more complex than just simple conversion.

The output will have 10 units, and the value for each will corelate the confidence of each item. The higher the confidence value of a specific unit means that the network thinks that it is that item.

The sum of all of the outputs will be 1. To do this, the output layer activation will be softmax.

Whenever the network does classification it will have the same number of output units as it does classes, and always use the softmax function

### Rectified Linear Unit (ReLU)

[Graph]

If the output is less than or equal to zero, the output will be zero. Otherwise the output will be equal to the input.

This gives the network the ability to solve nonlinear problems.

### Review

*Flattening* - The process of taking a 2d image to a 1d vector  
*ReLU* - An activation function that allows a network to solve nonlinear problems  
*Softmax* - A function that provides probability for each possible output class
*Classification* - A machine learning model used for distinguishing among two or more output categories